In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("../데이터자료/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_베이시스','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [2]:
df["1Y_베이시스대비"] =df["1Y_베이시스"]-df["1Y_베이시스"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()


,종가_ex,1Y_베이시스,국고1년,통안364일,Mid_ndf,M1_스왑포인트,1Y_베이시스대비,국고1년대비,통안364일대비,Mid_ndf대비,M1_스왑포인트대비
DateTime,,,,,,,,,,,
2012-08-01,1126.5,-76.0,2.82,2.80,1132.25,2.6,NaN,NaN,NaN,NaN,NaN
2012-08-02,1131.7,-74.0,2.79,2.77,1134.00,2.5,2.0,-0.03,-0.03,1.75,-0.1
2012-08-03,1134.8,-72.0,2.77,2.75,1138.00,2.4,2.0,-0.02,-0.02,4.00,-0.1
2012-08-06,1129.0,-74.0,2.78,2.76,1128.50,2.4,-2.0,0.01,0.01,-9.50,0.0
2012-08-07,1128.8,-73.0,2.77,2.75,1129.00,2.5,1.0,-0.01,-0.01,0.50,0.1


In [3]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns


Index(['종가_ex', '1Y_베이시스', '국고1년', '통안364일', 'Mid_ndf', 'M1_스왑포인트',
       '1Y_베이시스대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트대비',
       '전날 종가_ex'],
      dtype='object')

In [4]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns
df_Target = df["종가_ex"]
df_Data = df[['1Y_베이시스대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트','전날 종가_ex']]
df_Data.feature= df_Data.columns
df_Data.feature

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35672\864625744.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_베이시스대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트', '전날 종가_ex'], dtype='object')

In [17]:
from unicodedata import category
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
tf.random.set_seed(2022)

X_train, X_test, y_train, y_test = train_test_split(df_Data, df_Target,test_size=0.2, random_state=15 )
category = np.max(y_train)
print(X_train)

            1Y_베이시스대비  국고1년대비  통안364일대비  Mid_ndf대비  M1_스왑포인트  전날 종가_ex
DateTime                                                              
2015-05-11        1.0   -0.01      0.00      -6.45      1.00    1088.3
2020-02-04       -1.0    0.02      0.02      -4.85     -0.60    1195.0
2016-10-14        1.0    0.00      0.00       4.25      0.25    1135.9
2012-12-11       -1.0    0.00      0.00      -3.65      1.55    1079.0
2017-06-07        0.0    0.00      0.01       0.25     -0.55    1118.3
...               ...     ...       ...        ...       ...       ...
2015-02-27        0.0    0.00      0.00       2.25      1.40    1097.2
2021-04-08        2.0   -0.01      0.00       0.50      0.10    1116.3
2019-09-02       -1.0    0.03      0.02       1.30     -1.00    1211.2
2021-07-20       -1.0   -0.02     -0.02       9.55      0.35    1147.8
2022-06-24        0.0   -0.01      0.02       2.20     -0.60    1301.8

[1966 rows x 6 columns]


In [13]:
X_train, X_test, y_train, y_test = train_test_split(df_Data, df_Target,test_size=0.2, random_state=15 )
df_Data.info()
# optimizer (최적화 함수 설정)
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
# 최적의 성능을 내고, 30번동안 mse가 감소하는 경우가 없다면, 조기 종료 설정
# es = EarlyStopping(monitor='mse', mode='min', patience=30)
# 모형 mse가 가장 적은 model을 저장하는 부분
# mc = ModelCheckpoint('best_model.h5', monitor='mse', mode='min', save_best_only=True)

# 모델 형성 (input은 feature 2개, loss는 MSE로 설정, 반복회수 50번 시행)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape = (2,)))
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse'])
model.fit(x_train,y_train,epochs=50, batch_size = 10) #, callbacks=[es, mc]) 

# 모델 예측값 dataset 형성
pred = pd.concat([test, pd.DataFrame(model.predict(x_test), index = test.index)], axis =1)

# 모델 결과 출력
plt.plot(pred[0], label='KOSPI predicted')
plt.plot(pred['^KS11'], label='KOSPI actual')
plt.title('KOSPI predict by Samsung Price')
plt.legend()
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('KOSPI')
plt.show()

<class 'pandas.core.frame.DataFrame'>
Index: 2458 entries, 2012-08-03 to 2022-07-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   1Y_베이시스대비  2458 non-null   float64
 1   국고1년대비     2458 non-null   float64
 2   통안364일대비   2458 non-null   float64
 3   Mid_ndf대비  2458 non-null   float64
 4   M1_스왑포인트   2458 non-null   float64
 5   전날 종가_ex   2458 non-null   float64
dtypes: float64(6)
memory usage: 199.0+ KB


In [12]:
train_seq = pad_sequences(X_train, maxlen=100)
val_seq = pad_sequences(y_train, maxlen=100)

ValueError: invalid literal for int() with base 10: '1Y_베이시스대비'